<a href="https://colab.research.google.com/github/AutoEssenceJM/proyecto_trading.ipynb/blob/main/proyecto_trading_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# MVP: Señales técnicas simples para SPY (S&P500 ETF)
# Pega todo esto en Google Colab y ejecútalo.
# =========================
# Celda A: instalar dependencias (ejecutar UNA vez)
!pip install yfinance ta mplfinance --quiet
# Si ya tenías pandas_ta instalado no es necesario desinstalarlo,
# pero si quieres limpiar: !pip uninstall -y pandas_ta


In [ ]:
import numpy as np
print("numpy version:", np.__version__)
print("¿numpy tiene atributo 'NaN'?", hasattr(np, 'NaN'))

In [ ]:
# Celda B: imports y flujo principal (pegalo en Colab después de instalar y reiniciar el runtime)

# 1) Imports básicos
import yfinance as yf              # para descargar precios históricos (Yahoo Finance)
import pandas as pd               # manipulación de dataframes
from datetime import datetime     # para obtener fecha actual
import mplfinance as mpf          # para graficar velas (candlesticks)

In [ ]:
# 2) Importar las clases de 'ta' que usaremos (RSI, MACD, SMA, ATR)
from ta.trend import SMAIndicator, MACD
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange



In [ ]:
# 3) Parámetros de ejemplo
ticker = "SPY"  # ETF que replica el S&P 500
start_date = "2018-01-01"
end_date = datetime.today().strftime("%Y-%m-%d")



In [ ]:

# 4) Descargar datos OHLCV desde yfinance
#    yfinance devuelve: Open, High, Low, Close, Adj Close, Volume
df = yf.download(ticker, start=start_date, end=end_date, progress=False)
print("Filas descargadas:", len(df))



In [ ]:
# 5) Limpieza mínima
df = df.dropna()  # eliminar filas vacías (si hay)
# Opcional: df = df.ffill()  # forward fill si hay huecos pequeños


In [ ]:
# 6) Calcular indicadores con 'ta'
# Forzamos a Serie 1-D usando .squeeze()
close_series = df["Close"].squeeze()

# SMA 50 y SMA 200
df["SMA50"] = SMAIndicator(close=close_series, window=50, fillna=True).sma_indicator()
df["SMA200"] = SMAIndicator(close=close_series, window=200, fillna=True).sma_indicator()

# RSI 14
df["RSI14"] = RSIIndicator(close=close_series, window=14, fillna=True).rsi()

# MACD
macd_obj = MACD(close=close_series, window_slow=26, window_fast=12, window_sign=9, fillna=True)
df["MACD"] = macd_obj.macd()
df["MACD_signal"] = macd_obj.macd_signal()
df["MACD_hist"] = macd_obj.macd_diff()

# ATR 14
df["ATR14"] = AverageTrueRange(high=df["High"].squeeze(),
                               low=df["Low"].squeeze(),
                               close=close_series,
                               window=14, fillna=True).average_true_range()





In [ ]:
# 7) Generar señales simples (ejemplo educativo)
#    Regla 1: Golden / Death cross (SMA50 vs SMA200)
df["signal_gc"] = 0
df.loc[df["SMA50"] > df["SMA200"], "signal_gc"] = 1    # tendencia alcista (SMA50 arriba)
df.loc[df["SMA50"] < df["SMA200"], "signal_gc"] = -1   # tendencia bajista

#    Regla 2: RSI extremos
df["signal_rsi"] = 0
df.loc[df["RSI14"] < 30, "signal_rsi"] = 1    # posible compra (sobreventa)
df.loc[df["RSI14"] > 70, "signal_rsi"] = -1   # posible venta (sobrecompra)

#    Señal combinada: ejemplo sencillo
df["signal_combined"] = 0
df.loc[(df["signal_gc"] == 1) & (df["RSI14"] < 50), "signal_combined"] = 1   # compra
df.loc[(df["signal_gc"] == -1) & (df["RSI14"] > 50), "signal_combined"] = -1 # venta



In [ ]:
# 8) Exportar señales a CSV para revisar fuera de Colab
df_signals = df[["Close", "SMA50", "SMA200", "RSI14", "MACD", "MACD_signal", "ATR14",
                 "signal_gc", "signal_rsi", "signal_combined"]].copy()
df_signals.to_csv("spy_signals_ta.csv")
print("Señales guardadas en spy_signals_ta.csv")



In [ ]:
# Asegurar que las columnas son simples y numéricas
# Si el DataFrame tiene columnas MultiIndex, las aplastamos
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[0] for col in df.columns]

# Convertir columnas OHLCV a float
ohlcv_cols = ["Open", "High", "Low", "Close", "Volume"]
df[ohlcv_cols] = df[ohlcv_cols].astype(float)


In [ ]:
# 9) Graficar últimas 300 sesiones con mplfinance e incluir las SMA

# Creamos un nuevo DF con solo las últimas 300 filas
df_last = df.tail(300)

# Definimos los addplot usando df_last, no el df completo
apds = [
    mpf.make_addplot(df_last["SMA50"], panel=0),
    mpf.make_addplot(df_last["SMA200"], panel=0),
]

# Graficar
mpf.plot(df_last,
         type='candle',
         style='yahoo',
         addplot=apds,
         volume=True,
         title=f"{ticker} - Últimos 300 días")




In [ ]:
# 10) Mostrar las últimas filas con señales
display(df_signals.tail(20))

In [ ]:
# =========================
# Interpretación de señales
# =========================

alertas = []

# Tendencia
if df["SMA50"].iloc[-1] > df["SMA200"].iloc[-1]:
    if df["RSI14"].iloc[-1] > 50 and df["MACD"].iloc[-1] > df["MACD_signal"].iloc[-1]:
        alertas.append("🟢 Tendencia alcista con fuerza")
    else:
        alertas.append("🟢 Tendencia alcista moderada")
elif df["SMA50"].iloc[-1] < df["SMA200"].iloc[-1]:
    if df["RSI14"].iloc[-1] < 50 and df["MACD"].iloc[-1] < df["MACD_signal"].iloc[-1]:
        alertas.append("🔴 Tendencia bajista con fuerza")
    else:
        alertas.append("🔴 Tendencia bajista moderada")

# Sobrecompra / Sobreventa
if df["RSI14"].iloc[-1] > 70:
    alertas.append("⚠️ Alta probabilidad de corrección (RSI en sobrecompra)")
elif df["RSI14"].iloc[-1] < 30:
    alertas.append("⚠️ Posible rebote técnico (RSI en sobreventa)")

# Volumen alto
volumen_medio = df["Volume"].tail(20).mean()
if df["Volume"].iloc[-1] > volumen_medio * 1.5:
    alertas.append("📊 Volumen inusualmente alto (posible entrada de manos fuertes)")

# Cruce de medias
if df["SMA50"].iloc[-1] > df["SMA200"].iloc[-1] and df["SMA50"].iloc[-2] <= df["SMA200"].iloc[-2]:
    alertas.append("🚀 Golden Cross detectado (señal alcista fuerte)")
elif df["SMA50"].iloc[-1] < df["SMA200"].iloc[-1] and df["SMA50"].iloc[-2] >= df["SMA200"].iloc[-2]:
    alertas.append("💀 Death Cross detectado (señal bajista fuerte)")

# Mostrar alertas
print("\n📢 ALERTAS DE TRADING:")
for alerta in alertas:
    print(alerta)


In [ ]:
# ----------------------------
# Generador automático de niveles: entradas, stops y objetivos
# ----------------------------
import math

# Parámetros (ajustables)
lookback = 30            # ventana para max/min reciente (días)
atr_multiplier_stop = 3  # stop = entry - k * ATR (para largos); ajusta según apetito de riesgo
rr_targets = [2.0, 3.0]  # objetivos como múltiplos del riesgo (R:R)
pullback_to_sma = True   # si True, sugerimos entrada en SMA50 como entrada conservadora

# Últimos valores
last_close = float(df["Close"].iloc[-1])
atr = float(df["ATR14"].iloc[-1])
sma50 = float(df["SMA50"].iloc[-1])
sma200 = float(df["SMA200"].iloc[-1])

# Swing high / low recientes (para fibs y resistencia/soporte)
recent_high = float(df["Close"].tail(lookback).max())
recent_low = float(df["Close"].tail(lookback).min())

# Función para redondear niveles de precio (opcional, para que se vean "limpios")
def round_price(x, tick=0.01):
    # redondea a tick (ej. 0.01 para acciones con centavos)
    return float(math.floor(x / tick) * tick)

# Generar niveles para LARGO (si la tendencia lo permite)
trade_levels = {}

if sma50 > sma200:
    # Tendencia alcista -> sugerencias de LONG
    # Opción agresiva: entrada en mercado (precio actual)
    entry_aggressive = last_close

    # Opción conservadora: esperar pullback a SMA50 o a soporte reciente
    entry_conservative = sma50 if pullback_to_sma else recent_low

    # Stops basados en ATR
    stop_aggressive = entry_aggressive - atr_multiplier_stop * atr
    stop_conservative = entry_conservative - atr_multiplier_stop * atr

    # Targets por R:R usando la distancia entry->stop
    risk_aggr = entry_aggressive - stop_aggressive
    risk_cons = entry_conservative - stop_conservative

    targets_aggr = [entry_aggressive + rr * risk_aggr for rr in rr_targets]
    targets_cons = [entry_conservative + rr * risk_cons for rr in rr_targets]

    # Targets alternativos usando extensión fib desde recent_low -> recent_high
    # (solo si recent_high > recent_low)
    fib_targets = []
    if recent_high > recent_low:
        move = recent_high - recent_low
        # extensiones comunes: 1.0 (máximo), 1.272, 1.618
        fib_mults = [1.0, 1.272, 1.618]
        fib_targets = [recent_high + m * move for m in fib_mults]

    trade_levels["long"] = {
        "trend": "bull",
        "entry_aggressive": round_price(entry_aggressive),
        "stop_aggressive": round_price(stop_aggressive),
        "targets_aggressive": [round_price(t) for t in targets_aggr],
        "entry_conservative": round_price(entry_conservative),
        "stop_conservative": round_price(stop_conservative),
        "targets_conservative": [round_price(t) for t in targets_cons],
        "fib_targets": [round_price(t) for t in fib_targets],
        "recent_low": round_price(recent_low),
        "recent_high": round_price(recent_high),
        "atr": round_price(atr)
    }
else:
    trade_levels["long"] = {"trend": "not_bull", "note": "No se recomienda entrada larga porque SMA50 <= SMA200"}

# Generar niveles para CORTO (si la tendencia lo permite)
if sma50 < sma200:
    entry_aggressive_s = last_close
    entry_conservative_s = sma50 if pullback_to_sma else recent_high

    stop_aggr_s = entry_aggressive_s + atr_multiplier_stop * atr
    stop_cons_s = entry_conservative_s + atr_multiplier_stop * atr

    risk_aggr_s = stop_aggr_s - entry_aggressive_s
    risk_cons_s = stop_cons_s - entry_conservative_s

    targets_aggr_s = [entry_aggressive_s - rr * risk_aggr_s for rr in rr_targets]
    targets_cons_s = [entry_conservative_s - rr * risk_cons_s for rr in rr_targets]

    fib_targets_s = []
    if recent_high > recent_low:
        move = recent_high - recent_low
        fib_mults = [1.0, 1.272, 1.618]
        fib_targets_s = [recent_low - m * move for m in fib_mults]

    trade_levels["short"] = {
        "trend": "bear",
        "entry_aggressive": round_price(entry_aggressive_s),
        "stop_aggressive": round_price(stop_aggr_s),
        "targets_aggressive": [round_price(t) for t in targets_aggr_s],
        "entry_conservative": round_price(entry_conservative_s),
        "stop_conservative": round_price(stop_cons_s),
        "targets_conservative": [round_price(t) for t in targets_cons_s],
        "fib_targets": [round_price(t) for t in fib_targets_s],
        "recent_low": round_price(recent_low),
        "recent_high": round_price(recent_high),
        "atr": round_price(atr)
    }
else:
    trade_levels["short"] = {"trend": "not_bear", "note": "No se recomienda entrada corta porque SMA50 >= SMA200"}

# Mostrar resultados (ejemplo)
import json
print("\n🔎 Niveles sugeridos (auto):")
print(json.dumps(trade_levels, indent=2))
